<a href="https://colab.research.google.com/github/ganeshred/NLPAuthorshipAttributionNLM/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def preprocess(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

# Example usage:
text = "I really enjoyed the movie, it was fantastic!"
inputs = preprocess(text)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']


In [4]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from torch.utils.data import Dataset, DataLoader
class AuthorshipAttribution(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = preprocess(text)
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        return input_ids, attention_mask, label


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('/content/gdrive/MyDrive/Task3.csv')

In [8]:
df.head(20)

,Generation,label,label_int
0,Flint man facing 10 dog fighting and drug deal...,grover,5
1,the 30 most troubling lines from donald trump ...,ctrl,0
2,Wakayama rafting is a type of traditional raft...,instructgpt,7
3,The movies that predicted an outbreak like cor...,grover,5
4,companies that pay taxes are allowed to use ta...,pplm,8
5,1 A free press is essential to a functioning d...,instructgpt,7
6,s Saturday rally1 Research the venue Examine a...,instructgpt,7
7,The COVID 19 pandemic is an ongoing pandemic o...,instructgpt,7
8,my search for the boy in a child abuse video m...,gpt2,3
9,trump obeys seinfeld s law of use 5 7 16 9 20 ...,xlnet,10


In [9]:
df = df.dropna()

In [10]:
import torch
from torch.utils.data import DataLoader, Dataset

class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = preprocess(text)
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        return input_ids, attention_mask, label


In [11]:
texts = df['Generation'].tolist()
ltoi = {l: i for i, l in enumerate(df['label'].unique())}

In [12]:
ltoi

{'grover': 0,
 'ctrl': 1,
 'instructgpt': 2,
 'pplm': 3,
 'gpt2': 4,
 'xlnet': 5,
 'gpt': 6,
 'xlm': 7,
 'human': 8,
 'gpt3': 9,
 'fair': 10}

In [13]:
df['label1'] = df['label'].apply(lambda y: ltoi[y])


In [14]:
labels = df['label1'].tolist()

In [15]:
len(labels)

11718

In [16]:
# labels[6241]
# labels[6242]
# labels[6243]

In [17]:
labels

[0,
 1,
 2,
 0,
 3,
 2,
 2,
 2,
 4,
 5,
 6,
 7,
 8,
 1,
 0,
 3,
 5,
 4,
 2,
 4,
 5,
 9,
 9,
 6,
 7,
 5,
 10,
 2,
 5,
 2,
 1,
 8,
 6,
 1,
 10,
 3,
 9,
 10,
 8,
 0,
 2,
 2,
 0,
 3,
 3,
 6,
 6,
 3,
 7,
 9,
 7,
 10,
 0,
 9,
 10,
 1,
 9,
 9,
 0,
 4,
 8,
 7,
 10,
 8,
 3,
 2,
 7,
 8,
 2,
 5,
 3,
 3,
 6,
 9,
 6,
 1,
 9,
 3,
 3,
 6,
 10,
 0,
 4,
 4,
 8,
 6,
 7,
 5,
 2,
 9,
 8,
 2,
 0,
 9,
 3,
 4,
 1,
 3,
 1,
 7,
 0,
 2,
 2,
 8,
 5,
 0,
 7,
 6,
 2,
 6,
 5,
 0,
 5,
 10,
 0,
 10,
 0,
 0,
 7,
 9,
 0,
 4,
 10,
 5,
 0,
 7,
 8,
 3,
 7,
 3,
 10,
 2,
 5,
 1,
 2,
 7,
 7,
 5,
 10,
 10,
 9,
 9,
 9,
 3,
 3,
 8,
 6,
 3,
 8,
 1,
 6,
 7,
 8,
 1,
 0,
 3,
 8,
 10,
 1,
 3,
 10,
 0,
 2,
 2,
 1,
 10,
 7,
 10,
 1,
 3,
 10,
 0,
 8,
 1,
 1,
 7,
 1,
 6,
 10,
 1,
 8,
 2,
 9,
 4,
 8,
 10,
 6,
 10,
 3,
 3,
 4,
 4,
 0,
 6,
 3,
 6,
 10,
 7,
 7,
 1,
 5,
 4,
 6,
 8,
 4,
 5,
 6,
 3,
 4,
 10,
 10,
 0,
 9,
 6,
 2,
 7,
 2,
 8,
 3,
 8,
 4,
 5,
 5,
 2,
 2,
 2,
 5,
 2,
 4,
 1,
 0,
 3,
 10,
 5,
 1,
 8,
 8,
 2,
 3,
 9,
 7,
 6,
 3,
 8,

In [18]:
dataset = SentimentDataset(texts, labels)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

class TransformerClassifier(nn.Module):
    def __init__(self, pretrained_model_name, num_classes):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(pretrained_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
        self.classifier = nn.Linear(self.transformer.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids, attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(last_hidden_state)
        return logits


In [20]:
import torch.nn as nn
from transformers import AdamW
from tqdm import tqdm

model = TransformerClassifier(pretrained_model_name='bert-base-uncased', num_classes=11)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in 

In [21]:
labels[6242]

2

In [22]:
for epoch in range(5):
    running_loss = 0.0
    correct_predictions = 0

    for input_ids, attention_mask, labels in tqdm(dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(predicted == labels).item()

    epoch_loss = running_loss / len(dataset)
    epoch_acc = correct_predictions / len(dataset)
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

100%|██████████| 1465/1465 [17:47<00:00,  1.37it/s]


Epoch 1 Loss: 0.0509 Acc: 0.8491


100%|██████████| 1465/1465 [17:45<00:00,  1.38it/s]


Epoch 2 Loss: 0.0161 Acc: 0.9542


 10%|▉         | 145/1465 [01:46<16:12,  1.36it/s]


KeyboardInterrupt: ignored

In [23]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/BERT_task3.pt')

In [24]:
def evaluate(model, dataloader):
    model.eval()
    device = next(model.parameters()).device
    running_loss = 0.0
    correct_predictions = 0
    class_correct = [0] * 11
    class_total = [0] * 11

    with torch.no_grad():
        for input_ids, attention_mask, labels in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(predicted == labels).item()

            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += int(predicted[i] == label)
                class_total[label] += 1

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions / len(dataloader.dataset)
    classwise_acc = [class_correct[i] / class_total[i] if class_total[i] > 0 else 0 for i in range(11)]
    return epoch_loss, epoch_acc, classwise_acc


In [25]:
evaluate(model, dataloader)

(0.003314529006868891,
 0.9922341696535245,
 [0.9943714821763602,
  1.0,
  0.9670433145009416,
  0.9868667917448405,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9971751412429378,
  0.9690431519699813])

In [26]:
ltoi

{'grover': 0,
 'ctrl': 1,
 'instructgpt': 2,
 'pplm': 3,
 'gpt2': 4,
 'xlnet': 5,
 'gpt': 6,
 'xlm': 7,
 'human': 8,
 'gpt3': 9,
 'fair': 10}